# Chapter 2

In [29]:
from operator import attrgetter
from recordclass import recordclass

Node = recordclass('Node', ('data', 'next'))

class LinkedList(object):
    
    def __init__(self, *items):
        
        node = None
        for item in items[::-1]:
            node = Node(item, node)
        self.head = node
        
    def __iter__(self):
        
        node = self.head
        while node is not None:
            yield node
            node = node.next
        
    def __str__(self):
        
        data = attrgetter('data')
        return ', '.join(map(str, map(data, self)))

In [31]:
ll = LinkedList(*list(range(10)))
print(ll)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9


## Question 1

In [35]:
def remove_duplicates(ll):

    prev = None
    items = set()
    node = ll.head
    while node is not None:
        if node.data in items:
            prev.next = node = node.next
        else:
            items.add(node.data)
            prev = node
            node = node.next
    return ll
ll = LinkedList(0, 1, 2, 3, 2, 1, 0)
print(ll)
print(remove_duplicates(ll))

0, 1, 2, 3, 2, 1, 0
0, 1, 2, 3


In [36]:
ll = LinkedList(*([0, 1, 2, 3, 2, 1, 0] * 3))
print(ll)
def remove_duplicates2(ll):
    head = ll.head
    while head is not None:
        prev, node = head, head.next
        while node is not None:
            if node.data == head.data:
                prev.next = node = node.next
            else:
                prev, node = node, node.next
        head = head.next
    return ll
print(remove_duplicates2(ll))

0, 1, 2, 3, 2, 1, 0, 0, 1, 2, 3, 2, 1, 0, 0, 1, 2, 3, 2, 1, 0
0, 1, 2, 3


In [37]:
%timeit -n 1000 remove_duplicates(LinkedList(0, 1, 2, 3, 2, 1, 0))
%timeit -n 1000 remove_duplicates2(LinkedList(0, 1, 2, 3, 2, 1, 0))

1000 loops, best of 3: 5.17 µs per loop
1000 loops, best of 3: 5.45 µs per loop


## Question 2

In [39]:
def nth_to_the_last(ll, n):
    
    i = 0
    n_th = node = ll.head
    while node is not None:
        if i > n:
           n_th = n_th.next 
        node = node.next
        i += 1
    if i < n:
        return None
    else:
        return n_th.data
    
ll = LinkedList(*list(range(6)))
print(ll)
print(nth_to_the_last(ll, 0))
print(nth_to_the_last(ll, 1))
print(nth_to_the_last(ll, 2))
print(nth_to_the_last(ll, 7))

0, 1, 2, 3, 4, 5
5
4
3
None


In [41]:
def nth_to_the_last2(ll, n):
    
    def nth_to_the_last2(node, n, first=False):
        if node is None:
            return -1
        
        v = nth_to_the_last2(node.next, n)
        if isinstance(v, int):
            if v + 1 == n:
                if first:
                    return node.data
                else:
                    return node
            else:
                if first:
                    raise IndexError("index out of range")
                else:
                    return v + 1
        else:
            if first:
                return v.data
            else:
                return v
    return nth_to_the_last2(ll.head, n, first=True)

print(nth_to_the_last2(ll, 0))
print(nth_to_the_last2(ll, 2))
print(nth_to_the_last2(ll, 5))

5
3
0


In [43]:
%timeit -n 1000 nth_to_the_last(ll, 5)
%timeit -n 1000 nth_to_the_last2(ll, 5)

1000 loops, best of 3: 757 ns per loop
1000 loops, best of 3: 2.56 µs per loop


## Question 2.3
Delete a given node from a linked list

In [45]:
def delete_node(node):
    if node.next is not None:
        next = node.next
        node.data, node.next = next.data, next.next
ll = LinkedList(*list(range(7)))
print(ll)
delete_node(ll.head.next.next)
print(ll)

0, 1, 2, 3, 4, 5, 6
0, 1, 3, 4, 5, 6


## Question 2.4
Partition a linked list

In [54]:
def partition_linked_list(ll, value):

    left_head = left_tail = None
    right_head = right_tail = None
    node = ll.head
    while node is not None:
        if node.data < value:
            if left_head is None:
                left_head = left_tail = node
            else:
                left_tail.next = left_tail = node
        else:
            if right_head is None:
                right_head = right_tail = node
            else:
                right_tail.next = right_tail = node
        node.next, node = None, node.next
    ll.head = left_head
    left_tail = right_head
    return ll
        
    
ll = LinkedList(0, 1, 2, 3, 3, 2, 1, 0)
print(ll)
print(partition_linked_list(ll, 2.5))

0, 1, 2, 3, 3, 2, 1, 0
0, 1, 2, 2, 1, 0


In [21]:
def partition_linked_list(ll, value):
    left = right = None
    def append(to, node, left=False):
        if left and to is None:
            ll.head = node
        if to is not None:
            to.next = node
        return node
    node = ll.head
    
    while node is not None:
        next, node.next = node.next, None
        
        if node.data == value:
            center = append(left, node, left=True)
        elif node.data < value:
            left = append(left, node, left=True)
        else:
            right = append(right, node)
        node = next

        return ll

ll = LinkedList(Node(2.5, Node(1, Node(2, Node(3, Node(2, Node(1, Node(0, None))))))))
print(ll)
print(partition_linked_list(ll, 2.5))

2.5 -> 1 -> 2 -> 3 -> 2 -> 1 -> 0 -> 
2.5 -> 


## Question 2.5
Sum up numbers represented by linked lists

In [25]:
def sum_linked_lists(a_list, b_list):
    
    t_list = LinkedList()
    a = a_list.head
    b = b_list.head
    c = 0
    while a is not None or b is not None:
        total = a.data + b.data
           
        t 
        if t_list.head is None:


SyntaxError: unexpected EOF while parsing (<ipython-input-25-4c852edcbcd2>, line 6)